In [41]:
import pyximport
pyximport.install()

(None, None)

In [42]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [43]:
import utils
import utils_opt
import sknn_opt
import sknn 
import pandas as pd
import numpy as np

In [44]:
FOLDER_TRAIN = 'data/train/short_ver/'
FOLDER_TEST = 'data/test/'
OUTPUT_FOLDER = 'output/'

In [62]:
%time train, test = utils.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.3066082000732422s
 -- filtered in: 0.40578508377075195s
 -- actions:  664712
 -- items:  170089
 -- lists:  10000
CPU times: user 339 ms, sys: 97.3 ms, total: 437 ms
Wall time: 462 ms


In [61]:
rec = sknn.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [47]:
%time rec.train(train, test)

CPU times: user 444 ms, sys: 57.9 ms, total: 501 ms
Wall time: 501 ms


In [48]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

In [49]:
tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

In [50]:
rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 

,track_id,confidence
0,1921,2.228768
1,802,1.623090
2,1244,1.087304
3,2398,1.041252
4,801,1.032605
...,...,...
495,6744,0.210092
496,7731,0.210040
497,8402,0.209968
498,65,0.209734


### Attempted optimized approach

In [ ]:
# %lprun -f rec.train rec.train(train, test)

In [63]:
%time train, test = utils_opt.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.25600290298461914s
 -- filtered in: 0.35828089714050293s
(' -- actions: ', 664712)
(' -- items: ', 170089)
(' -- lists: ', 10000)
CPU times: user 345 ms, sys: 92.8 ms, total: 437 ms
Wall time: 412 ms


In [52]:
rec = sknn_opt.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [53]:
%time rec.train(train, test)

CPU times: user 348 ms, sys: 57.1 ms, total: 405 ms
Wall time: 404 ms


In [54]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

In [55]:
tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

In [56]:
rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 

,track_id,confidence
0,1921,2.228768
1,802,1.623090
2,1244,1.087304
3,2398,1.041252
4,801,1.032605
...,...,...
495,6744,0.210092
496,7731,0.210040
497,8402,0.209968
498,65,0.209734
